In [1]:
from keras.models import load_model
import glob
import numpy as np
from collections import Counter
import pandas as pd

Using TensorFlow backend.


In [2]:
new_model = load_model('Fri_May_17_22-52-09_2019_2dcnn_spec_.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
def get_subarrays(row):
    return [row[pos:pos + 3,] for pos in range(0, len(row),3)]

In [4]:
labels = {'Saxophone': 0,
 'Glockenspiel': 1,
 'Cello': 2,
 'Knock': 3,
 'Gunshot_or_gunfire': 4,
 'Hi-hat': 5,
 'Laughter': 6,
 'Flute': 7,
 'Telephone': 8,
 'Bark': 9,
 'Scissors': 10,
 'Gong': 11,
 'Microwave_oven': 12,
 'Shatter': 13,
 'Harmonica': 14,
 'Bass_drum': 15,
 'Oboe': 16,
 'Bus': 17,
 'Tambourine': 18,
 'Keys_jangling': 19,
 'Electric_piano': 20,
 'Clarinet': 21,
 'Fireworks': 22,
 'Meow': 23,
 'Double_bass': 24,
 'Cough': 25,
 'Acoustic_guitar': 26,
 'Violin_or_fiddle': 27,
 'Snare_drum': 28,
 'Squeak': 29,
 'Finger_snapping': 30,
 'Writing': 31,
 'Trumpet': 32,
 'Drawer_open_or_close': 33,
 'Cowbell': 34,
 'Tearing': 35,
 'Fart': 36,
 'Chime': 37,
 'Burping_or_eructation': 38,
 'Computer_keyboard': 39,
 'Applause': 40}

inverted_mapping = {v:k for k,v in labels.items()}

In [12]:
def get_predicted_label(row):
    try:
        data = np.load(path + row + '.npy').T
        data_slice = get_subarrays(data)
        xs = []
        for array_set in data_slice:
            if array_set.shape == (3,128):  # The network take input of shape (3, 128, 1)
                xs.append(array_set)
        features = np.asarray(xs)
        xs_reshape = features.reshape(features.shape[0], 3, 128, 1)
        pred_labels = [inverted_mapping.get(x) for x in new_model.predict_classes(xs_reshape)] # going to return a prediction for each of the 33
        label = Counter(pred_labels).most_common(1)[0][0]
        return label
    except Exception as e:
        return "none"

# Get predictions for Test_Audios files

In [6]:
test_npys = glob.glob('test_npy/' +'*.npy')
print(len(test_npys))
print(test_npys[0])

9394
test_npy/8319139c.wav.npy


In [7]:
test_audios = glob.glob('audio_test/' +'*.wav')
len(test_audios)

9400

In [8]:
file_names = [x.split("/")[1] for x in test_audios]
print(len(file_names))
df = pd.DataFrame(file_names)
df.rename(columns={0: "fname"},inplace=True)

df.head()

9400


,fname
0,8319139c.wav
1,003995fa.wav
2,9c80c2dc.wav
3,93dbba2e.wav
4,45fbe67a.wav


In [13]:
path = 'test_npy/'
df['label'] = df['fname'].map(get_predicted_label)
df.shape

(9400, 2)

In [14]:
df.head()

,fname,label
0,8319139c.wav,Hi-hat
1,003995fa.wav,Squeak
2,9c80c2dc.wav,Bus
3,93dbba2e.wav,Cello
4,45fbe67a.wav,Fireworks


# Get predictions for Train_Audio files

In [15]:
train = pd.read_csv('train.csv')
train.head()
train.shape

(9473, 3)

In [16]:
path = 'single_label_npy2/'
train['pre_label'] = train['fname'].map(get_predicted_label)

In [32]:
train.groupby('pre_label').count()

,fname,label,manually_verified
pre_label,,,
Acoustic_guitar,355,355,355
Applause,343,343,343
Bark,185,185,185
Bass_drum,162,162,162
Burping_or_eructation,147,147,147
Bus,388,388,388
Cello,242,242,242
Chime,105,105,105
Clarinet,268,268,268


In [37]:
train_clean = train[train.pre_label !="none"]
train_clean.shape

(9464, 4)

In [38]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.metrics import classification_report
y_true = train_clean.label.to_list()
y_pred = train_clean.pre_label.to_list()
confusion_matrix = confusion_matrix(y_true, y_pred)
confusion_matrix

array([[229,   0,   1, ...,   0,   6,   2],
       [  0, 247,   5, ...,   0,   1,   0],
       [  0,   3, 138, ...,   0,   0,   5],
       ...,
       [  8,   2,   6, ..., 118,  15,   7],
       [  4,   0,   0, ...,   1, 274,   1],
       [  0,   5,   1, ...,   0,   0, 151]])

In [48]:
print(classification_report(y_true, y_pred))

                       precision    recall  f1-score   support

      Acoustic_guitar       0.65      0.76      0.70       300
             Applause       0.72      0.82      0.77       300
                 Bark       0.75      0.58      0.65       238
            Bass_drum       0.91      0.49      0.64       300
Burping_or_eructation       0.99      0.69      0.81       210
                  Bus       0.25      0.88      0.39       109
                Cello       0.77      0.62      0.69       300
                Chime       0.67      0.61      0.64       115
             Clarinet       0.88      0.78      0.83       300
    Computer_keyboard       0.33      0.70      0.44       119
                Cough       0.72      0.57      0.64       243
              Cowbell       0.99      0.79      0.88       190
          Double_bass       0.69      0.77      0.73       300
 Drawer_open_or_close       0.37      0.59      0.45       158
       Electric_piano       0.62      0.59      0.61  